In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("CW2") \
    .getOrCreate()

sc = spark.sparkContext

In [2]:
sc

<SparkContext master=local[*] appName=CW2>

In [3]:
from time import time
from preprocessing.Preprocessor import Preprocessor

In [4]:
import os
os.path.abspath(os.getcwd())

'/Users/sachinsuri/CW - Sentiment Analysis (NLP)'

In [5]:
pp = Preprocessor(spark)

In [6]:
train = pp.to_bow('Cleaned Data/train.csv', min_frequency=275., remove_stop_words = True)
val =  pp.to_bow('Cleaned Data/val.csv')

In [7]:
#train = pp.balance_classes(train)

In [7]:
train.cache()
val.cache()

DataFrame[features: vector, label: int]

In [8]:
train.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1854,[0,1,3,12,1...|    2|
|(1854,[3,13,74,89...|    0|
|(1854,[0,9,10,19,...|    0|
|(1854,[4,8,36,184...|    2|
|(1854,[2,5,6,17,2...|    2|
+--------------------+-----+
only showing top 5 rows



In [9]:
val.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(1854,[8,14,16,21...|    0|
|(1854,[0,1,4,7,8,...|    2|
|(1854,[0,1,3,6,10...|    0|
|(1854,[0,2,3,6,7,...|    2|
|(1854,[0,1,2,4,5,...|    2|
+--------------------+-----+
only showing top 5 rows



In [10]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [11]:
lst_smooth = [0, 1.0, 100.0, 500, 1000]
lst_model = []
"""lst_model_transform = []"""


for i in lst_smooth:
    nb = NaiveBayes(smoothing=i, modelType="multinomial")
    model = nb.fit(train)
    lst_model.append(model)
    """train_preds_and_labels = model.transform(train).select('prediction', 'label')
    lst_model_transform.append(train_preds_and_labels)"""
    
    
"""nb = NaiveBayes(smoothing = 1.0, modelType="multinomial")
model = nb.fit(train)
train_fitting_check = model.transform(train).select('prediction', 'label')"""

'nb = NaiveBayes(smoothing = 1.0, modelType="multinomial")\nmodel = nb.fit(train)\ntrain_fitting_check = model.transform(train).select(\'prediction\', \'label\')'

In [12]:
print(lst_model)

[NaiveBayesModel: uid=NaiveBayes_7b4b61ea4b05, modelType=multinomial, numClasses=3, numFeatures=1854, NaiveBayesModel: uid=NaiveBayes_9045edfb5212, modelType=multinomial, numClasses=3, numFeatures=1854, NaiveBayesModel: uid=NaiveBayes_246724241068, modelType=multinomial, numClasses=3, numFeatures=1854, NaiveBayesModel: uid=NaiveBayes_6c125e7a81a8, modelType=multinomial, numClasses=3, numFeatures=1854, NaiveBayesModel: uid=NaiveBayes_e6a1cfe346f5, modelType=multinomial, numClasses=3, numFeatures=1854]


In [13]:
train_m = Metrics()
for i in lst_model_transform:
    train_m.f1(i)
    
"""from metrics.metrics import Metrics
train_m = Metrics()
train_m.f1(train_fitting_check)
train_m.accuracy(train_fitting_check)
train_m.confusion_matrix(train_fitting_check)
train_m.performance(train_fitting_check)"""

F1 score for given DataFrame: 72.7

Accuracy on given DataFrame: 72.6%

                Predictions
            negative    neutral     positive    
negative    17724       2215        8408        
neutral     2903        2546        4873        
positive    7042        5966        63093       
(True labels are on the side)

Class     | Precision | Recall    | F1-score
negative    64.1        62.5        63.3        
neutral     23.7        24.7        24.2        
positive    82.6        82.9        82.8        



In [14]:
#print(lst_model)

In [13]:
model_predictions = []

for i in lst_model:
    predictions = i.transform(val)
    model_predictions.append(predictions)
"""predictions = model.transform(val)"""

'predictions = model.transform(val)'

In [16]:
"""for i in model_predictions:
    evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="accuracy")
    accuracy = evaluator.evaluate(i)
    print("Test set accuracy = " + str(accuracy))"""

evaluator = MulticlassClassificationEvaluator(labelCol = "label", predictionCol="prediction", metricName = "accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.7243513521753712


In [17]:
val_m = Metrics()
val_fitting_check = model.transform(val).select('prediction', 'label')
val_m.f1(val_fitting_check)
val_m.accuracy(val_fitting_check)
val_m.confusion_matrix(val_fitting_check)
val_m.performance(val_fitting_check)

F1 score for given DataFrame: 72.5

Accuracy on given DataFrame: 72.4%

                Predictions
            negative    neutral     positive    
negative    5125        650         2492        
neutral     818         654         1343        
positive    2030        1708        17979       
(True labels are on the side)

Class     | Precision | Recall    | F1-score
negative    64.3        62.0        63.1        
neutral     21.7        23.2        22.4        
positive    82.4        82.8        82.6        



In [19]:
from metrics.metrics import Metrics
lst_evaluate_f1 = []
val_m = Metrics()
for i in model_predictions:
    val_fit_check = val_m.f1(i.select('prediction','label'))
    lst_evaluate_f1.append(val_fit_check)

F1 score for given DataFrame: 72.5

F1 score for given DataFrame: 72.5

F1 score for given DataFrame: 71.5

F1 score for given DataFrame: 60.7

F1 score for given DataFrame: 54.5



In [17]:
for i in lst_evaluate_f1:
    print(i)

DataFrame[prediction: double, label: int]
DataFrame[prediction: double, label: int]
DataFrame[prediction: double, label: int]
DataFrame[prediction: double, label: int]
DataFrame[prediction: double, label: int]
